# Data Annotation with Prediction Assistant
This notebook has the same capabilities as `data_processing.ipynb` except each frame is analyzed by a model and predictions are drawn onto the screen
Key commands

| Button | Description |
|--------|-------------|
| s | **Save** file w/ annotations |
| a | Move image to `to_annotate` |
| z | **Undo** most recent box |
| L click | **Draw** box |
| R click | **Highlight** boxes to delete |
| r | **Remove** highlighted boxes |
| 1 | Change light to **green** |
| 2 | Change light to **red** |
| 3 | Change light to **yellow** |


Required imports

In [1]:
import os
import sys
import shutil
from datetime import datetime

import cv2
from ultralytics import YOLO

sys.path.append(os.path.abspath('../src'))
import utils


Choose model for predictions

In [2]:
# Load model for predictions
curr_model = os.listdir("../models/current_assistant")[0]
model_path = f'../models/current_assistant/{curr_model}'
model =  YOLO(model_path)

Directories for retrieving, copying, or moving frames and videos

In [3]:
# Video folders
trimmed_video_dir = '../data/videos/trimmed'
processed_video_dir = '../data/videos/processed'

# Unprocessed frames
frame_dir = '../data/images/frames'

# Resized and annotated frames
processed_dir = "../data/images/processed"

# 1920x1080 original copies
original_dir = "../data/images/frames_original"

# Images to augment later
augmenting_dir = "../data/images/for_augmenting"

# Frames that are going to be used for training the newest annotation assistant
pretrain_dir = "../data/images/pretrain"

Gather screen information so that annotating data is a smoother experience

In [4]:
import screeninfo

# Get second monitor details
monitors = screeninfo.get_monitors()

# Two monitor setup
if len(monitors) > 1:
    second_monitor = monitors[1]
    x_offset, y_offset = second_monitor.x, second_monitor.y
    screen_width, screen_height = second_monitor.width, second_monitor.height

# Default to primary monitor
else:
    x_offset, y_offset = 0, 0
    screen_width, screen_height = monitors[0].width, monitors[0].height 

# 95% screen width and height
window_width = int(screen_width * 0.95)
window_height = int(screen_height * 0.95)

# Center window
window_x = x_offset + (screen_width - window_width) // 2
window_y = y_offset + (screen_height - window_height) // 2

Helper functions for **redrawing** and **editing** BBoxes

### Variables

In [5]:
# Exit labelling variable
exit = False

# Change model
change_model = False

# Standardized image width/height
IMAGE_WIDTH, IMAGE_HEIGHT = 768, 448

# Iterate over every n-th frame
STEP = 1

# Color mapping based on key presses
color_mapping = {
    ord("1"): (0, 255, 0),   # Green
    ord("2"): (0, 0, 255),   # Red
    ord("3"): (0, 255, 255), # Yellow
}

# Label mapping based on colors
label_mapping = {
    (0, 255, 0): "green_light",
    (0, 0, 255): "red_light",
    (0, 255, 255): "yellow_light"
}

# Class mapping based on label (for YOLO format that uses int instead of str)
class_mapping = {
    "green_light": 1,
    "red_light": 2,
    "yellow_light": 3
}

### Main Code
#### 1. Helper functions for redrawing and editing BBoxes
#### 2. Iterate over available frames

In [6]:
# Draws annotations onto copy of resized image then resets the cv2 frame (img_copy)
def redraw_bbox(annotations):
    global img, img_copy
    img = resized_img.copy()  # reset to the resized image
    for ann in annotations:
        cv2.rectangle(img, (int(ann["x1"]), int(ann["y1"])), (int(ann["x2"]), int(ann["y2"])), ann["color_code"], int(ann['thickness']))
    img_copy = img.copy()

# Based on mouse events, draws/deletes/edits BBoxes
def edit_bbox(event, x, y, flags, params):

    # right click to select the box the cursor is inside of
    if event == cv2.EVENT_RBUTTONDOWN:
        clickedBox = False
        for i, ann in enumerate(annotations):
            if x > min(ann['x1'], ann['x2']) and x < max(ann['x1'], ann['x2']) and y > min(ann['y1'], ann['y2']) and y < max(ann['y1'], ann['y2']):
                print(annotations[i])
                ann["thickness"] = 2
                redraw_bbox(annotations)
                clickedBox = True

        if not clickedBox:
            utils.reset_selection(annotations)
            redraw_bbox(annotations)

    global ix, iy, drawing, img_copy, img, current_color

    # Left clicking starts a drawing event if the user is not currently drawing
    if event == cv2.EVENT_LBUTTONDOWN and not drawing:  
        drawing = True # event status is drawing
        ix, iy = x, y # anchor point for the first corner of the rectangle
        img_copy = img.copy()  # reset copy when starting a new rectangle

    # When the cursor is moving and we are in drawing status display adjusted size of rectangle based on cursor location
    elif event == cv2.EVENT_MOUSEMOVE and drawing:  
        img_copy = img.copy()  # reset to avoid multiple overlapping rectangles
        cv2.rectangle(img_copy, (ix, iy), (x, y), current_color, 1) # drawing rectangle from ix, iy to current cursor position

    # Left click when we are already drawing places the rectangle where the cursor is located during the click
    elif event == cv2.EVENT_LBUTTONDOWN and drawing:  
        drawing = False # reset event status to not drawing
        cv2.rectangle(img, (ix, iy), (x, y), current_color, 1)  # draw on final image
        annotations.append({
            "x1": min(ix, x),
            "x2": max(ix, x),
            "y1": min(iy, y),
            "y2": max(iy, y),
            "color_code": current_color,
            "color": label_mapping[current_color],
            "class": class_mapping[label_mapping[current_color]],
            "thickness": 1
        }) # appends a map of values needed for documentation min/max x and y coordinates, color codes, colors, and class
        
        redraw_bbox(annotations)  # for view consistency



# Iterate over each file in the frame dir
files = os.listdir(frame_dir)
for i in range(0, len(files), 1):
    filename = files[i]
    
    # List to store dicts of annotations
    annotations = []
    if exit:
        break

    file_path = os.path.join(frame_dir, filename)

    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load image
        original_img = cv2.imread(file_path)  # keep original image
        if original_img is None:
            raise FileNotFoundError("Image not found. Check the file path.")
        
        # Resize the image to a uniform size
        resized_img = cv2.resize(original_img, (IMAGE_WIDTH, IMAGE_HEIGHT))

        # Initialize working images
        img = resized_img.copy()   # Active drawing image
        img_copy = img.copy()      # Image copy for real-time updates
        result = model(resized_img, verbose=False)[0]

        # add predictions to annotations
        for ann in result.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = ann
            annotations.append({
                "x1": int(min(x1, x2)),
                "x2": int(max(x1, x2)),
                "y1": int(min(y1, y2)),
                "y2": int(max(y1, y2)),
                "color_code": color_mapping[ord(str(int(class_id)))],
                "color": result.names[int(class_id)],
                "class": int(class_id),
                "thickness": 1
                }  
            )
            # draw BBox
            cv2.rectangle(img_copy, (int(x1), int(y1)), (int(x2), int(y2)), color_mapping[ord(str(int(class_id)))], 1)

        # Anchor variables
        ix, iy = -1, -1
        drawing = False
        current_color = (0, 255, 0)  # Default: Green

        # Prepare annotation file
        img_filename = os.path.basename(file_path)
        print(img_filename)
        print(os.path.splitext(img_filename)[0])
        text_filename = os.path.splitext(img_filename)[0] + ".txt"
        
        # Viewing annotation file
        viewing_annotation_path = f"../data/labels/viewing/viewing_{text_filename}"
        # Formatted annotation file
        yolo_annotations_path = f'../data/labels/formatted/{text_filename}'


        # Create window and set mouse callback
        window_name = f"Label Data: {filename}"
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)  # Allow resizing
        cv2.resizeWindow(window_name, window_width, window_height)  # Set to 95% of screen size
        cv2.moveWindow(window_name, window_x, window_y)  # Center it on the second monitor
        cv2.setMouseCallback(window_name, edit_bbox)

        # Display loop
        while True:
            cv2.imshow(window_name, img_copy)  # Show dynamic updates
            key = cv2.waitKey(10) & 0xFF
            
            # Press 'Esc' to exit
            if key == 27:
                exit = True
                break
            
            # Press 's' to save annotations
            elif key == ord("s"):

                # Open viewing text file, iterate over annotations and write to file
                with open(viewing_annotation_path, 'w') as viewing_file:
                    for annotation in annotations:
                        viewing_file.write(f"{annotation}\n")
                print(f"Viewing annotations saved to {viewing_annotation_path}")
                
                # Convert annotations to yolo format
                yolo_annotations = utils.viewing_to_yolo(annotations, IMAGE_WIDTH, IMAGE_HEIGHT)

                # Open yolo text file, iterate over annotations and write to file
                with open(yolo_annotations_path, 'w') as yolo_file:
                    for yolo_ann in yolo_annotations:
                        yolo_str = " ".join(map(str, yolo_ann))  # Convert each item to string and join with commas
                        yolo_file.write(f"{yolo_str}\n")  # Write formatted string to file
                print(f"YOLO annotations saved to {yolo_annotations_path}")


                # Open processed_dir and write resized image to the directory
                # os.makedirs(processed_dir, exist_ok=True)
                # processed_path = os.path.join(processed_dir, os.path.basename(file_path))
                # cv2.imwrite(processed_path, resized_img)
                # print(f"Moved {filename} -> {processed_path}")

                # updated to move to pretrain first
                os.makedirs(pretrain_dir, exist_ok=True)
                pretrain_path = os.path.join(pretrain_dir, os.path.basename(file_path))
                cv2.imwrite(pretrain_path, resized_img)
                print(f"Moved {filename} -> {pretrain_path}")

                os.makedirs(original_dir, exist_ok=True)
                original_path = os.path.join(original_dir, os.path.basename(file_path))
                shutil.move(file_path, original_path)
                break


            # Press 'a' to move frame to data/images/for_augmenting
            elif key == ord('a'):
                for_augmenting_path = os.path.join(augmenting_dir, os.path.basename(file_path))
                shutil.move(file_path, for_augmenting_path)
                print(f"Moved {filename} -> {for_augmenting_path}")
                break
                
            # Change rectangle color based on number key
            elif key in color_mapping:  
                current_color = color_mapping[key]
                print(f"Class changed to: {label_mapping[color_mapping[key]]}")
            
            # Press 'r' to remove bbox drawn by model assistant
            elif key == ord("r"):
                utils.remove_bbox(annotations)
                redraw_bbox(annotations)
                
            # Press 'z' to undo last rectangle
            elif key == ord("z") and annotations: 
                annotations.pop()  # remove last rectangle
                redraw_bbox(annotations)  # reset image and redraw remaining rectangles
                print("Last rectangle removed!")

            elif key == ord("c"):
                change_model = True

        cv2.destroyAllWindows()
        
        if len(os.listdir(pretrain_dir)) >= 200:
            date_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            utils.train_assistant_model(date_time)
            utils.dump_data()
        if change_model:
            curr_model = os.listdir("../models/current_assistant")[0]
            model_path = f'../models/current_assistant/{curr_model}'
            model =  YOLO(model_path)
            change_model = False
cv2.destroyAllWindows()




20250222_152541M_000274.jpg
20250222_152541M_000274
Viewing annotations saved to ../data/labels/viewing/viewing_20250222_152541M_000274.txt
YOLO annotations saved to ../data/labels/formatted/20250222_152541M_000274.txt
Moved 20250222_152541M_000274.jpg -> ../data/images/pretrain\20250222_152541M_000274.jpg
20250222_152541M_000275.jpg
20250222_152541M_000275
Viewing annotations saved to ../data/labels/viewing/viewing_20250222_152541M_000275.txt
YOLO annotations saved to ../data/labels/formatted/20250222_152541M_000275.txt
Moved 20250222_152541M_000275.jpg -> ../data/images/pretrain\20250222_152541M_000275.jpg
20250222_152541M_000276.jpg
20250222_152541M_000276
Viewing annotations saved to ../data/labels/viewing/viewing_20250222_152541M_000276.txt
YOLO annotations saved to ../data/labels/formatted/20250222_152541M_000276.txt
Moved 20250222_152541M_000276.jpg -> ../data/images/pretrain\20250222_152541M_000276.jpg
20250222_152541M_000277.jpg
20250222_152541M_000277
Viewing annotations save